# Data Cleaning and Transformation
Denne sektion handler om at rense og transformere data ved hjælp af pandas. Vi vil dække følgende emner:
- Håndtering af manglende værdier
- Navngivning og omdøbning af kolonner
- Indeksering og udvælgelse af data
- Maskering og betinget udvælgelse
- Filtrering og sortering af data
- Data transformation (f.eks. normalisering, one-hot encoding)
- Gruppere og aggregere data
- Arbejde med datoer og tider


In [15]:
import pandas as pd

#trees = pd.read_csv('trees.csv', header=None)              # Hvis man ikke vil have kolonne navne
#trees = pd.read_csv('trees.csv', names=['id', 'højde'])    # Hvis man har en liste af kolonne navne
trees = pd.read_csv('trees.csv')

trees.info() # Grundig info om strukturen på datafram, kan også kaldes på en serie

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   afd_bv        237 non-null    object 
 1   plot_id       237 non-null    object 
 2   tree_no       237 non-null    int64  
 3   species_id    237 non-null    int64  
 4   art           237 non-null    object 
 5   height_m      237 non-null    object 
 6   h2            10 non-null     float64
 7   dbh_cm        237 non-null    object 
 8   DHM_Z         237 non-null    object 
 9   lat_long      237 non-null    object 
 10  measure_date  237 non-null    object 
 11  plot_area     237 non-null    object 
 12  qa_flag       159 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 24.2+ KB


- **Non-null Count:** Antallet af ikke-manglende værdier i hver kolonne. f. eks. er der kun 10 entries af h2.
- **dtypes: float64(1), int64(2), object(10):** Antallet af kolonner af hver datatype.
- **memory usage: 24.2+ KB:** Den mængde hukommelse, som DataFrame bruger.

In [16]:
trees.head()

,afd_bv,plot_id,tree_no,species_id,art,height_m,h2,dbh_cm,DHM_Z,lat_long,measure_date,plot_area,qa_flag
0,2a,P1,1,20,Eg,"11,4",NaN,37.7,"52,88","55.750358,12.400765",2024-07-26,723.1 m2,OK
1,2a,P1,2,20,Eg,1900 cm,NaN,14.4,40.13,"55.750000,12.400287",2024-04-29,882.2,ok
2,2a,P1,3,40,Aer,16.9,NaN,14.2 cm,41.61,"55.750655,12.399541",2024-06-13,671.3,ok
3,2a,P1,4,20,Eg,26.8,NaN,12.8 cm,"65,04","55.750720,12.399568",2024-06-27,729.1,OK
4,2a,P1,5,40,Ær,26.4,NaN,16.4,"59,26","55.750401,12.400270",2024-06-03,680.7,OK


I dette dataset vil jeg gerne omdøbe kolonnerne height_m og h2 til height_1_m og height_2_m for at gøre dem mere beskrivende. samt DHM_Z til DHM_Z_m.

- Først definere jeg et dictionary, der kortlægger de gamle kolonnenavne til de nye.
- Derefter bruger jeg `rename` metoden på DataFrame til at omdøbe kolonnerne ved at angive `mapper` argumentet med mit dictionary og `axis=1` for at indikere, at jeg omdøber kolonner (i stedet for rækker, som bruger `axis=0`).

In [19]:
column_mapper = {'height_m': 'height_1_m',
                 'h2': 'height_2_m',
                 'DHM_Z': 'DHM_Z_m'}
trees = trees.rename(mapper = column_mapper, axis = 1)

trees.head()

,afd_bv,plot_id,tree_no,species_id,art,height_1_m,height_2_m,dbh_cm,DHM_Z_m,lat_long,measure_date,plot_area,qa_flag
0,2a,P1,1,20,Eg,"11,4",NaN,37.7,"52,88","55.750358,12.400765",2024-07-26,723.1 m2,OK
1,2a,P1,2,20,Eg,1900 cm,NaN,14.4,40.13,"55.750000,12.400287",2024-04-29,882.2,ok
2,2a,P1,3,40,Aer,16.9,NaN,14.2 cm,41.61,"55.750655,12.399541",2024-06-13,671.3,ok
3,2a,P1,4,20,Eg,26.8,NaN,12.8 cm,"65,04","55.750720,12.399568",2024-06-27,729.1,OK
4,2a,P1,5,40,Ær,26.4,NaN,16.4,"59,26","55.750401,12.400270",2024-06-03,680.7,OK


Jeg har opdaget ud fra `info()` at kolonnen h2 (nu height_2_m) har mange manglende værdier (kun 10 non-null ud af 20). Da det ikke er nok til at lave en meningsfuld analyse, vil jeg fjerne denne kolonne fra DataFrame.

- Jeg bruger `drop` metoden på DataFrame og angiver kolonnenavnet 'height_2_m' samt `axis=1` for at indikere, at jeg vil droppe en kolonne (i stedet for en række, som bruger `axis=0`).
- Her kunne jeg have lavet en liste af kolonner, hvis jeg ville droppe flere kolonner på én gang.

In [20]:
trees = trees.drop(['height_2_m'], axis=1)

# Jeg omdøber height_1_m igen da tallet ikke giver mening længere
trees = trees.rename(mapper={'height_1_m': 'height_m'}, axis=1)
trees.head()

,afd_bv,plot_id,tree_no,species_id,art,height_m,dbh_cm,DHM_Z_m,lat_long,measure_date,plot_area,qa_flag
0,2a,P1,1,20,Eg,"11,4",37.7,"52,88","55.750358,12.400765",2024-07-26,723.1 m2,OK
1,2a,P1,2,20,Eg,1900 cm,14.4,40.13,"55.750000,12.400287",2024-04-29,882.2,ok
2,2a,P1,3,40,Aer,16.9,14.2 cm,41.61,"55.750655,12.399541",2024-06-13,671.3,ok
3,2a,P1,4,20,Eg,26.8,12.8 cm,"65,04","55.750720,12.399568",2024-06-27,729.1,OK
4,2a,P1,5,40,Ær,26.4,16.4,"59,26","55.750401,12.400270",2024-06-03,680.7,OK
